<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex06notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex06notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## 主成分分析 (1)
----

**主成分分析** (Principal Component Analysis, PCA) は，多変量（多次元）のデータが与えられたときに，そのデータが含む情報をなるべく損なわずにより低次元のデータに変換するための多変量解析手法です．
$D$個の変数から成る（$D$次元の）データが与えられたときに，それを$H \leq D$ 個の変数で表す（$H$次元に変換する）ことを目的とします．この変換によってデータの次元数が削減されますので，このような操作は **次元削減** (Dimensionaliry Reduction) と呼ばれることもあります．主成分分析は，次元削減のための手法の一つです．

---
### 準備

#### インポート

In [ ]:
# いろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

#### データの準備

次のコードセルでは，「SSDSE-家計消費」というデータセットを読み込んで前処理を行うコードを実行します．
データについての説明も表示します．

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/DiningOutSpendingData.py
from DiningOutSpendingData import DiningOutSpendingData
data = DiningOutSpendingData()
kinki5 = data.kinki5()
tohoku6 = data.tohoku6()
data.info()

次のコードセルを実行すると，最初の5件のデータを表示します．

In [ ]:
# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1
# DataFrame を入手して最初の5件を表示
dfFeature = data.getDataFrame()
dfFeature.head(5)

---
### やりたいこと



上でやっていた準備作業では，47都道府県の県庁所在市の13種類の外食支出金額（ひとりあたり，単位は円）の数値が得られました．このデータでは47の市ごとにその特徴が13個の変数で表されていますので，13次元のデータといえます．
これだけ変数の数が多いと，それらの数値を眺めただけで各市の傾向を理解するのは困難です．そこで，「このようなデータをもっと低次元で（少ない数の変数で）表す」ことを考えます．主成分分析は，このようなことを行うための手法です．




In [ ]:
# 13種類の支出金額ならべた配列を作る
X = data.getArray()
print(X[:5, :])
print(X.shape)

次のセルを実行すると，上記のデータに主成分分析を適用して，13次元のデータを2次元に変換したものが得られます．

<!---
ここでは [scikit-learn](https://scikit-learn.org/) という Python 向けの機械学習ライブラリを使っています．scikit-learn については授業の範囲外ですので，コードの中身の理解は必須ではありません．
cf. [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
--->
ここでは，[statsmodels](https://www.statsmodels.org/) という Python 向け統計分析ライブラリを使っています．
cf. [statsmodels.multivariate.pca.PCA](https://www.statsmodels.org/dev/generated/statsmodels.multivariate.pca.PCA.html)

In [ ]:
# 主成分分析を利用して X を2次元に次元削減した Y を得る（標準化あり）
from statsmodels.multivariate.pca import PCA
pca = PCA(X, ncomp=2, standardize=True, demean=True, normalize=False)
Y = pca.scores
print(Y[:5, :])
print(Y.shape)

`X` は13次元のデータだったので `shape` が `(47, 13)` でしたが，それを変換して得られた `Y` の `shape` は `(47, 2)` です．各市の特徴を2つの数値で表したものとなっています．

2次元のデータなので，散布図に描けます．描いてみるとこんなん：

In [ ]:
# Y の散布図
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[kinki5, 0], Y[kinki5, 1], label='kinki5')
ax.scatter(Y[tohoku6, 0], Y[tohoku6, 1], label='tohoku6')
ax.set_xlim(-4.5, 4.5)
ax.set_ylim(-4.5, 4.5)
ax.set_aspect('equal')
ax.axhline(0, color='gray') # x軸
ax.axvline(0, color='gray') # y軸
ax.legend()
for n in range(len(Y)):
    plt.annotate(f'{n}', (Y[n, 0]+0.1, Y[n, 1]+0))
plt.show()

横軸が `Y` の0列目の値，縦軸が 1 列目の値です．各点に付された番号は，行番号（0から始まる）に対応しています．

これを見ると，ほげ市とふが市は点が近いところにある → 似た傾向にありそうだ，といったことを読み取れそうですね．また，ちゃんとした説明は後にしますが，得られた値を見て各市の特徴をつかむことも容易になります．

この図では，わかりやすくするために，近畿5府県と東北6県の点の色を変えてみています．それぞれ，元の13次元のデータはこんなんです：

In [ ]:
# 近畿5府県
dfFeature.iloc[data.kinki5(), :]

In [ ]:
# 東北6県
dfFeature.iloc[data.tohoku6(), :]

In [ ]:
# お好きなところをどぞ
dfFeature.iloc[3, :]  # 行番号で
#dfFeature.query('City == "札幌市"')  # City 列の値で

ここでは13次元のデータを2次元にしたので散布図を描いてみせましたが，いつでもどんなデータでも2次元にする，という話ではないことに注意してください．13次元のデータを4次元にしたり，1万次元のデータを10次元にしたり，データの性質と分析の目的によって次元数は様々です．

---
### 1次元に次元削減する場合の主成分分析

↑に書いたことをもう一度書いておきます：

> **主成分分析** (Principal Component Analysis, PCA) は，多変量（多次元）のデータが与えられたときに，そのデータが含む情報をなるべく損なわずにより低次元のデータに変換するための多変量解析手法です．
$D$個の変数から成る（$D$次元の）データが与えられたときに，それを$H \leq D$ 個の変数で表す（$H$次元に変換する）ことを目的とします．この変換によってデータの次元数が削減されますので，このような操作は **次元削減** (Dimensionaliry Reduction) と呼ばれることもあります．主成分分析は，次元削減のための手法の一つです．


ここでは，まずは $H = 1$ の場合について考えます．

#### 考え方

主成分分析では，「データが含む情報をなるべく損なわずにより低次元のデータに変換する」ということを，「**変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）する**」ということで達成しようとします．
ここでは，元のデータの次元数 $D = 2$ の場合を例として，これを1次元に次元削減する場合を図に描きながら考えてみましょう．



下図の左は，とある2次元のデータを散布図に描いたものです．二つの破線はそれぞれ，横軸と縦軸の値の平均を表しています．
このデータの値を，それらの平均が $\pmb{0}$ になるように平行移動させると，散布図は下図の右のようになります．
以降，このように平均が $\pmb{0}$ になるように平行移動させたデータの集合を
$$
\{ \pmb{x}_n \in R^{D} | n = 1, 2, \ldots, N\} \qquad (1)
$$
と表すことにします．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig01.png">


このようなデータを1次元にする線形変換は，$D$次元列ベクトル $\pmb{u}$ をひとつ定めて，下図のように$D$次元列ベクトル $\pmb{x}$ を1つの実数値 $y$ に変換する操作と考えられます．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig02.png">

この図の左の赤い直線は，ベクトル $\pmb{u}$ の向きを表しています．個々のデータ $\pmb{x}$ に対して，「その点がこの赤い直線上に落とす影」を考えて，この直線上でのその影の位置を $y$ とおくと，右図に示すように

$$
y = \Vert\pmb{x}\Vert\cos\theta \qquad (2)
$$

となります．$\theta$ は $\pmb{x}$ と $\pmb{u}$ の成す角，$\|\pmb{x}\|$ は $\pmb{x}$ のノルム（大きさ）です．
$\pmb{u}\cdot\pmb{x} = \|\pmb{u}\|\|\pmb{x}\|\cos\theta$ であることから，上の式は，次のように書き換えられます．

$$
y = \frac{\pmb{u}\cdot\pmb{x}}{\|\pmb{u}\|} \qquad (3)
$$

与えられたデータに対して，ベクトル $\pmb{u}$ を適当に定めてやれば，この式によって $D$ 次元のデータを1つの実数値に変換できます．

ここで，ベクトル $\pmb{u}$  として，向きが同じでノルムのみ異なる二つの候補があったとしてみましょう．その場合，得られる $y$ の値はどちらを用いても同じとなります．つまり，ここで考えている変換では，ベクトル $\pmb{u}$ の向きのみが重要ということになります．
そのため，ベクトル $\pmb{u}$ のノルムは $1$ に限定して（つまり，$\pmb{u}$ は単位ベクトルに限定して）考えることができます．この場合，$\Vert\pmb{u}\Vert = 1$ ですので，上の式はより簡単に

$$
y = \pmb{u}\cdot\pmb{x} \qquad (4)
$$

と表せます．

以上より，$D$次元の単位ベクトル $\pmb{u}$ を適当に定めれば，$y_n = \pmb{u}\cdot\pmb{x}_n$ という式で1次元への次元削減ができることが分かりました．
では，$\pmb{u}$ はどのように定めればよいでしょうか？

主成分分析の目的は，「変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）」することでした．$\pmb{u}$の向きを変えると，変換後の値 $y_n$ の分散の大きさも変化します．
以下の図(a), (b), (c) は，3種類の $\pmb{u}$ で2次元のデータを1次元に変換した様子と，変換後の値の分散を示しています．
右図は横軸に $y_n$ の値をとっており，`variance` の値が $y_n$ の分散です．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig03.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig04.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig05.png">

図を見ると，データの散らばりが大きい方向を $\pmb{u}$ が向いたときに，変換後の分散が大きくなっていることがわかります．
直感的な説明になりますが，変換後の値だけを見て元のデータの様子を想像するなら，分散が大きい方が，個々の値の違いがわかりやすくてうれしいですね．

ここまで，データを1次元に次元削減する場合限定で，主成分分析の考え方を説明してきました．
次のセクションでは，このような主成分分析の問題をきちんと定式化して，その解がどのようなものかを説明します．

#### 主成分分析の問題設定とその解（1次元にする場合）

---

［主成分分析の問題設定（1次元にする場合）］

$N$個の$D$次元ベクトルから成るデータ

$$
\{ \pmb{x}_n \in R^{D}  | n = 1, 2, \ldots, N\} \qquad (5)
$$

が与えられるとする．このデータの平均は $\mathbf{0}$ である，つまり，

$$
\frac{1}{N}\sum_{n=1}^{N}\pmb{x}_{n} = \pmb{0} \qquad (6)
$$

と仮定する（平均が $\pmb{0}$でない場合は，平均を求めてそれをデータから差し引いたものをあらためて $\pmb{x}_n$ とおけばよい）．
このとき，$D$ 次元単位ベクトル $\pmb{u}$ を用いて，$D$ 次元ベクトル $\pmb{x}$ を次のように1つの実数値 $y_n$ に変換することを考える．

$$
y_n = \pmb{u}\cdot \pmb{x}_n \quad (n = 1, 2, \ldots, N) \qquad (7)
$$

ベクトル $\pmb{u}$ を，$\Vert\pmb{u}\Vert = 1$ の任意のベクトルの中で $y_n$の分散が最大となるように定めたい．

---


これが主成分分析の問題設定です．
この問題の解，つまり，$y_n$ の分散を最大にする $\pmb{u}$ は，「**データ $\{\pmb{x}_n\}$ の分散共分散行列を $V$ とするとき，その最大の固有値に対応する単位固有ベクトル（ノルムが1の固有ベクトル）**」となります（導出過程は後述）．

ここで，**分散共分散行列** （「共分散行列」ともいう） は，次のように定められます：

$\pmb{x}_n$($n=1,2,\ldots,N$)を $D$ 次元列ベクトル（$D\times 1$ 行列）とするとき，その分散共分散行列 $V$ は

$$
V = \frac{1}{N}\sum_{n=1}^{N}(\pmb{x}_{n} - \bar{\pmb{x}})(\pmb{x}_{n} - \bar{\pmb{x}})^{\top} \qquad (8)
$$

という $D\times D$ 行列です（$\bar{\pmb{x}}$ は $\pmb{x}_n$ の平均）．$V$ の $(i,j)$ 要素 $V_{i,j}$ は

$$
V_{i,j} = \frac{1}{N}\sum_{n=1}^{N}(x_{n,i}-\bar{x}_{i})(x_{n,j}-\bar{x}_{j})  \qquad (i, j = 1, 2, \ldots, D) \qquad (9)
$$

という式で表されますが，これは $\pmb{x}_n$ の $i$ 番目の要素と $j$ 番目の要素の間の **共分散** です（注）．

今の問題設定では，$\pmb{x}_n$ の平均は $\bar{\pmb{x}} = \pmb{0}$ と仮定していますので，

$$
V = \frac{1}{N}\sum_{n=1}^{N}\pmb{x}_{n}\pmb{x}_{n}^{\top} \qquad (10)
$$

となります．
まとめると，ここに示した主成分分析の解のベクトル $\pmb{u}$ は，次のステップで求められます．

1. データから $V$ を計算する
1. その固有値固有ベクトルを求める
1. 最大の固有値に対応する単位固有ベクトルを $\pmb{u}$ とする

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注： ここでは「分散共分散行列」や「共分散」についてはこれ以上は深入りしません．「データ分析」を履修したひとは，「相関係数」の話で登場していますので，そのときの資料が参考になるかもしれません
（<a href="https://www-tlab.math.ryukoku.ac.jp/wiki/?Data/2024#ex05">2024年度「データ分析」ex05 散布図と相関</a>）．
</span>


#### ［発展］主成分分析の問題の解の導出（1次元にする場合）

上記の解の導出過程の概略を以下に示します．

仮定より $\pmb{x}_n$ の平均が $\pmb{0}$ なので，$y_n$ の平均を $\bar{y}$ とおくと

$$
\bar{y} = \frac{1}{N}\sum_{n=1}^{N}y_n = \frac{1}{N}\sum_{n=1}^{N}\pmb{u}\cdot\pmb{x}_{n} = \pmb{u}\cdot\left( \frac{1}{N}\sum_{n=1}^{N}\pmb{x}_n\right) = \pmb{u}\cdot\pmb{0} = 0 \qquad (11)
$$

が成り立ちます．そのため，$y_n$ の分散は

$$
\frac{1}{N}\sum_{n=1}^{N}(y_n - \bar{y})^2 = \frac{1}{N}\sum_{n=1}^{N}y_n^2 \qquad (12)
$$

と表せます．この式は，次のように変形できます．

$$
\begin{aligned}
\frac{1}{N}\sum_{n=1}^{N}y_n^2 &= \frac{1}{N}\sum_{n=1}^{N} (\pmb{u}\cdot\pmb{x}_n)(\pmb{u}\cdot\pmb{x}_n) \qquad (13)\\
&= \frac{1}{N}\sum_{n=1}^{N} \left(\pmb{u}^{\top}\pmb{x}_n\right) \left(\pmb{u}^{\top}\pmb{x}_n\right)^{\top} \qquad (14)\\
&= \frac{1}{N}\sum_{n=1}^{N} \pmb{u}^{\top}\pmb{x}_n\pmb{x}_n^{\top} \pmb{u} \qquad (15)\\
&= \pmb{u}^{\top}\left( \frac{1}{N}\sum_{n=1}^{N}\pmb{x}_n\pmb{x}_n^{\top}\right) \pmb{u} = \pmb{u}^{\top}V\pmb{u} \qquad (16)
\end{aligned}
$$
<!---
ただし，
$$
V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}
$$
は，$\mathbf{x}_n$ の平均が $\mathbf{0}$ であるため，$\mathbf{x}_n$の分散共分散行列に一致します．
--->


いま求めたいのは，$y_n$ の分散 $\pmb{u}^\top V\pmb{u}$ を最大にするベクトル $\pmb{u}$ です．ただし，$\pmb{u}$ は単位ベクトルに限定されていますので，$\Vert\pmb{u}\Vert = 1$ という条件があります．
このような制約条件付きの最適化問題を解く定番の手法は，「ラグランジュの未定乗数法」です（注）．
この問題の場合，ラグランジュ乗数を $\lambda$ として，

$$
L = \pmb{u}^{\top}V\pmb{u} - \lambda(\pmb{u}^{\top}\pmb{u} - 1) \qquad (17)
$$

とおけば，$\frac{\partial L}{\partial \pmb{u}} = \pmb{0}$ を満たす $\pmb{u}$ が解の候補となります．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「ラグランジュの未定乗数法」は，大学初年次の微積分で学んでいる...かも．興味のあるひとは数学の参考書を調べてね．
</span>

ここで，

$$
\frac{\partial L}{\partial \pmb{u}} = 2V\pmb{u} - 2\lambda\pmb{u} \qquad (18)
$$

なので（注），解は

$$
V\pmb{u} = \lambda\pmb{u} \qquad (19)
$$

を満たさねばなりません．この式より，$V$ の単位固有ベクトルが解の候補であることがわかります．この式を $L$ の式に代入すると，

$$
L = \lambda\pmb{u}^\top\pmb{u} - \lambda\cdot 0 = \lambda \qquad (20)
$$

となります．解の候補は $V$ の単位固有ベクトルであり，そのときの $L$ の値が固有値に等しいということですので，$L$ を最大にする解は，$V$の最大固有値に対する固有ベクトルとなります．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: $\frac{\partial L}{\partial \pmb{u}}$ は，「$L$を $\pmb{u}$ の各要素で偏微分したものをならべたベクトル」です．
</span>

---
### 2次元以上に次元削減する場合の主成分分析


> **主成分分析** (Principal Component Analysis, PCA) は，多変量（多次元）のデータが与えられたときに，そのデータが含む情報をなるべく損なわずにより低次元のデータに変換するための多変量解析手法です．
$D$個の変数から成る（$D$次元の）データが与えられたときに，それを$H \leq D$ 個の変数で表す（$H$次元に変換する）ことを目的とします．この変換によってデータの次元数が削減されますので，このような操作は **次元削減** (Dimensionaliry Reduction) と呼ばれることもあります．主成分分析は，次元削減のための手法の一つです．


$2 \leq H \leq D$ の場合の主成分分析について説明します．

$D$ 次元列ベクトル $\pmb{x}$ に対して，$H$ 個の $D$ 次元列ベクトル $\pmb{u}_1, \pmb{u}_2, \ldots,  \pmb{u}_H$ を横にならべた行列を

$$
U = \begin{pmatrix} \pmb{u}_1 & \pmb{u}_2 & \cdots & \pmb{u}_H \end{pmatrix} \qquad (21)
$$

とおきます．$U$ は $D\times H$ 行列です．このとき，$\pmb{x}$ に左から

$$
U^{\top}
=
\begin{pmatrix} \pmb{u}_1 & \pmb{u}_2 & \cdots & \pmb{u}_H \end{pmatrix}^{\top}
=
\begin{pmatrix}
\pmb{u}_1^{\top} \\ \pmb{u}_2^{\top} \\ \vdots \\ \pmb{u}_H^{\top}
\end{pmatrix} \qquad (22)
$$

をかけたものを $\pmb{y}$ とおくと，

$$
\pmb{y} = U^{\top}\pmb{x}
=
\begin{pmatrix}
\pmb{u}_1^{\top} \\ \pmb{u}_2^{\top} \\ \vdots \\ \pmb{u}_H^{\top}
\end{pmatrix} \pmb{x}
=
\begin{pmatrix}
\pmb{u}_1^{\top}\pmb{x} \\ \pmb{u}_2^{\top}\pmb{x} \\ \vdots \\ \pmb{u}_H^{\top}\pmb{x}
\end{pmatrix}
=
\begin{pmatrix}
\pmb{u}_1\cdot\pmb{x} \\ \pmb{u}_2\cdot\pmb{x} \\ \vdots \\ \pmb{u}_H\cdot\pmb{x}
\end{pmatrix}  \qquad (23)
$$

は $H$ 次元ベクトルになります．したがって，何らかの方法で $H$ 個の $D$ 次元ベクトル $\pmb{u}_1, \pmb{u}_2, \ldots, \pmb{u}_H$ を定めてそれらをならべた行列 $U$ を作り，式$(23)$ の $\pmb{y} = U^{\top}\pmb{x}$ という計算を行うことで，$D$ 次元のデータを $H$ 次元に変換することができます．

<img width=800 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/dimreduction_new.png">


では，$\pmb{u}_1, \ldots, \pmb{u}_H$ はどのように定めればよいのでしょうか．
いろいろな考え方ができるのですが，主成分分析では，「データが含む情報をなるべく損なわずにより低次元のデータに変換する」ために，「**変換後のデータの分散の総和がなるべく大きくなるように（元のデータの分散を保つように）**（注）」します．$H = 1$ の場合，これを実現するには，「データの分散共分散行列の最大固有値に対応する単位固有ベクトル」を選べばよい，ということでした．

$H = 2$ の場合は，$\pmb{u}_1$を「データの分散共分散行列の最大固有値に対応する単位固有ベクトル」とした上で，$\pmb{u}_2$ を「データの分散のうち $\pmb{u}_1$ で説明しきれなかった分を最もよく説明できる向き」にとります．
これは，$\pmb{u}_2$ を，「$\pmb{u}_1$ に直交しかつ大きさが $1$ のベクトルの中で，$y_2 = \pmb{u}_2\cdot\pmb{x}$ の分散が最も大きくなるものとする」ことを意味し，$\pmb{u}_2$ を「データの分散共分散行列の2番目に大きな固有値に対応する固有ベクトル」の向きにとればよいことがわかっています（数学的にきちんとした議論は省略します）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 一般の（$H \geq 2$の）場合，$\pmb{y}$ の各要素間の共分散が $0$ になるように（分散共分散行列が対角行列になるように）かつ各要素の分散（分散共分散行列の対角要素）の和が最大になるようにします．
</span>

下図は，$D=3$ の場合を図示しています．左の図のピンクのベクトルは $\pmb{u}_1$ を表しており，このデータはこのベクトルの向きに最も分散が大きくなっています．
中央の緑色の平面は，原点を通りかつピンクのベクトルと直交しています．$\pmb{u}_2$ は，この平面内の様々な向きのベクトルたち（?で示されているもの）の中から選ぶことになります．このとき，下図右のように，この緑の平面上でデータ点を眺めたときに分散が最も大きくなる方向を選べば，$y_2$の分散を最大化することになります．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig06.png">

$H \geq 3$ の場合も，$\pmb{u}_2$ と同じように一つずつベクトルを追加していくと考えることができます．すなわち，$\pmb{u}_3$ は，$\pmb{u}_{1}, \pmb{u}_2$ の両方に直交する単位ベクトルの中で分散最大の方向のものとし，$\pmb{u}_4$ は $\pmb{u}_{1}, \pmb{u}_2, \pmb{u}_3$ の全てに直交し...，という具合です．そして，結論として，次のことがいえます．

---

［主成分分析］

$N$個の$D$次元列ベクトルから成るデータ

$$
\{ \pmb{x}_n  | n = 1, 2, \ldots, N\}
$$

を，$D \times H$ 行列 $U$ によって

$$
\mathbf{y}_n = U^{\top}\mathbf{x}_n
$$

と $H$ 次元に変換することを考える．ただし，$\{\pmb{x}_n\}$ の平均は $\pmb{0}$ とする．
このとき，変換後のデータの分散の総和がなるべく大きくなるように（元のデータの分散を保つように）したいならば，$U$ を次のように構成すればよい．

1. データの分散共分散行列の固有値と固有ベクトルを求める．固有値を $\lambda_1 \geq \lambda_2 \geq \lambda_3 \geq \cdots \geq \lambda_D$ とし，それぞれに対応する単位固有ベクトルを $\pmb{u}_1, \pmb{u}_2, \ldots, \pmb{u}_D$ とおく（注）．
2. $\pmb{u}_1, \pmb{u}_2, \ldots, \pmb{u}_H$ をならべて， $D\times H$ 行列 $U$ を $U = \begin{pmatrix} \pmb{u}_1 & \pmb{u}_2 & \cdots & \pmb{u}_H \end{pmatrix}$ とする．

---

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 分散共分散行列は対称行列であるため，固有値は必ず実数となります．また，$0$ 以上となることもいえます．
</span>


---
### 主成分と主成分スコア

$D$ 個の変数から成るベクトル $\pmb{x}$ を上記の手順で $H$ 個の変数から成るベクトル $\pmb{y}$ に変換したとき，得られる $H$ 個の変数のうち $h$ 番目のもの（$\pmb{y}$ の $h$ 番目の要素，$h = 1, 2, \ldots, H$）を，**第 $h$ 主成分** と呼びます．
上記の説明から分かるように，番号の小さい主成分ほど元のデータの分散をよく表す変数となっています．
直感的な言い方になりますが，番号の小さい主成分ほど元のデータの散らばりをよく表す重要な変数といえます．
また，一つのデータを変換して得られる$H$個の主成分のうち，第 $h$ 主成分の変数の値のことを，**第 $h$ 主成分スコア** と呼びます．


